In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


def import_tweets(filename, header = None):
    #import data from csv file via pandas library
    tweet_dataset = pd.read_csv(filename,header = header, encoding='Latin-1', low_memory=False, index_col=False)
    #the column names are based on sentiment140 dataset provided on kaggle
    tweet_dataset.columns = ['target','id','date','flag','user','text']
    #delete 3 columns: flags,id,user, as they are not required for analysi
    for i in ['flag','id','user','date']: del tweet_dataset[i] # or tweet_dataset = tweet_dataset.drop(["id","user","date","user"], axis = 1)
    #in sentiment140 dataset, positive = 4, negative = 0; So we change positive to 1
    tweet_dataset.target = tweet_dataset.target.replace(4,1)
    return tweet_dataset

def preprocess_tweet(tweet):
	#Preprocess the text in a single tweet
	#arguments: tweet = a single tweet in form of string
	#convert the tweet to lower case

	str(tweet)
	tweet.lower()
	#convert all urls to sting "URL "
	tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
	#convert all @username to "AT_USER "
	tweet = re.sub('@[^\s]+','AT_USER', tweet)
	#correct all multiple white spaces to a single white space
	tweet = re.sub('[\s]+', ' ', tweet)
	#convert "#topic" to just "topic"
	tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
	return tweet




In [3]:
my_df = import_tweets("training_1600000_processed_noemoticon.csv")
print("File read")

my_df['text'] = my_df['text'].apply(preprocess_tweet)
print("Preprocessing done")

# csv = 'clean_tweet.csv'
# my_df = pd.read_csv(csv,index_col=0)
my_df.head()

File read


Preprocessing done


,target,text
0,0,"AT_USER URL - Awww, that's a bummer. You shoul..."
1,0,is upset that he can't update his Facebook by ...
2,0,AT_USER I dived many times for the ball. Manag...
3,0,my whole body feels itchy and like its on fire
4,0,"AT_USER no, it's not behaving at all. i'm mad...."


In [4]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
target    1600000 non-null int64
text      1600000 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [5]:
x = my_df.text
y = my_df.target

In [6]:
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.1, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [7]:
print( "Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print ("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print ("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 1440000 entries with 49.97% negative, 50.03% positive
Validation set has total 80000 entries with 50.27% negative, 49.73% positive
Test set has total 80000 entries with 50.18% negative, 49.82% positive


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
# tvec1 = TfidfVectorizer(sublinear_tf=True,stop_words="english")  # we need to give proper stopwords list for better performance

# tvec1.fit(x_train)
x_train_tfidf = tvec1.fit_transform(x_train)
print("Done")

Done


In [10]:
print("Done1")
x_validation_tfidf = tvec1.transform(x_validation)
# x_validation_tfidf = tvec1.transform(x_validation).toarray()

print("Done2")

Done1


Done2


In [11]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train_tfidf, y_train)

C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 1min 11s
Parser   : 279 ms


In [13]:
clf.score(x_validation_tfidf, y_validation)

0.8221

In [14]:
clf.score(x_train_tfidf, y_train)

0.8416604166666667

I will first start by loading required dependencies. In order to run Keras with TensorFlow backend, you need to install both TensorFlow and Keras.

In [15]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


The structure of below NN model has 100,000 nodes in the input layer, then 64 nodes in a hidden layer with Relu activation function applied, then finally one output layer with sigmoid activation function applied. There are different types of optimizing techniques for neural networks, and different loss function you can define with the model. Below model uses ADAM optimizing, and binary cross entropy loss.

ADAM is an optimization algorithm for updating the parameters and minimizing the cost of the neural network, which is proved to be very effective. It combines two methods of optimization: RMSProp, Momentum. Again, I will focus on sharing the result I got from my implementation, but if you want to understand properly how ADAM works, I strongly recommend the "deeplearning.ai" course by Andrew Ng. He explains the complex concept of neural network in a very intuitive way.

Before I feed the data and train the model, I need to deal with one more thing. Keras NN model cannot handle sparse matrix directly. The data has to be dense array or matrix, but transforming the whole training data Tfidf vectors of 1.5 million to dense array won't fit into my RAM. So I had to define a function, which generates iterable generator object so that it can be fed to NN model. Note that the output should be a generator class object rather than arrays, this can be achieved by using "yield" instead of "return". 

In [16]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0

In [18]:
%%time
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=100000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=2, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/2


    1/45000 [..............................] - ETA: 30:13:55 - loss: 0.6935 - acc: 0.4688

    2/45000 [..............................] - ETA: 16:39:53 - loss: 0.6936 - acc: 0.4062

    3/45000 [..............................] - ETA: 12:08:02 - loss: 0.6934 - acc: 0.4583

    4/45000 [..............................] - ETA: 9:53:58 - loss: 0.6931 - acc: 0.4844 

...

11160/45000 [======>.......................] - ETA: 2:17:32 - loss: 0.4377 - acc: 0.7968

11161/45000 [======>.......................] - ETA: 2:17:32 - loss: 0.4377 - acc: 0.7968

11162/45000 [======>.......................] - ETA: 2:17:32 - loss: 0.4377 - acc: 0.7968

11163/45000 [======>.......................] - ETA: 2:17:31 - loss: 0.4377 - acc: 0.7968

KeyboardInterrupt: 

It looks like the model had the best validation accuracy after 2 epochs, and after that, it fails to generalize so validation accuracy slowly decreases, while training accuracy increases. But if you remember the result I got from logistic regression (train accuracy: 84.19%, validation accuracy: 82.91%), you can see that the above neural network failed to outperform logistic regression in terms of validation.

Let's see if normalizing inputs have any effect on the performance.

In [42]:
from sklearn.preprocessing import Normalizer
norm = Normalizer().fit(x_train_tfidf)
x_train_tfidf_norm = norm.transform(x_train_tfidf)
x_validation_tfidf_norm = norm.transform(x_validation_tfidf)

In [46]:
%%time
model_n = Sequential()
model_n.add(Dense(64, activation='relu', input_dim=100000))
model_n.add(Dense(1, activation='sigmoid'))
model_n.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_n.fit_generator(generator=batch_generator(x_train_tfidf_norm, y_train, 32),
                    epochs=2, validation_data=(x_validation_tfidf_norm, y_validation),
                    steps_per_epoch=x_train_tfidf_norm.shape[0]/32)

Epoch 1/5
48878/48878 [==============================] - 5471s 112ms/step - loss: 0.4081 - acc: 0.8142 - val_loss: 0.3941 - val_acc: 0.8253
Epoch 2/5
48878/48878 [==============================] - 5268s 108ms/step - loss: 0.3726 - acc: 0.8351 - val_loss: 0.3967 - val_acc: 0.8262
Epoch 3/5
48878/48878 [==============================] - 5110s 105ms/step - loss: 0.3558 - acc: 0.8456 - val_loss: 0.4189 - val_acc: 0.8139
Epoch 4/5
48878/48878 [==============================] - 5239s 107ms/step - loss: 0.3285 - acc: 0.8603 - val_loss: 0.4521 - val_acc: 0.8046
Epoch 5/5
48878/48878 [==============================] - 5162s 106ms/step - loss: 0.3036 - acc: 0.8736 - val_loss: 0.4698 - val_acc: 0.7978
CPU times: user 14h 4min 6s, sys: 3h 42min 59s, total: 17h 47min 6s
Wall time: 7h 17min 31s


By the look of the result, normalizing seems to have almost no effect on the performance. And it is at this point I realized that Tfidf is already normalized by the way it is calculated. TF (Term Frequency) in Tfidf is not absolute frequency but relative frequency, and by multiplying IDF (Inverse Document Frequency) to the relative term frequency value, it further normalizes the value in a cross-document manner.

If the problem of the model is a poor generalization, then there is another thing I can add to the model. Even though the neural network is a very powerful model, sometimes overfitting to the training data can be a problem. Dropout is a technique that addresses this problem. If you are familiar with the concept of ensemble model in machine learning, dropout can also be seen in the same vein. According to the research paper "Improving neural networks by preventing
co-adaptation of feature detectors" by Hinton et al. (2012), "A good way to reduce the error on the test set is to
average the predictions produced by a very large number of different networks. The standard way to do this is to train many separate networks and then to apply each of these networks to the test data, but this is computationally expensive during both training and testing. Random dropout makes it possible to train a huge number of different networks in a reasonable time." https://arxiv.org/pdf/1207.0580.pdf

Dropout is simulating as if we train many different networks and averaging them by randomly omitting hidden nodes with a certain probability throughout the training process. With Keras, this can be easily implemented just by adding one line to your model architecture. Let's see how the model performance changes with 20% dropout rate. (*I will gather all the results and present them with a table at the end.)

In [21]:
model1 = Sequential()
model1.add(Dense(64, activation='relu', input_dim=100000))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model1.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
48878/48878 [==============================] - 5019s 103ms/step - loss: 0.4101 - acc: 0.8139 - val_loss: 0.3951 - val_acc: 0.8252 - ac - ETA: 2s - los
Epoch 2/5
48878/48878 [==============================] - 5022s 103ms/step - loss: 0.3769 - acc: 0.8335 - val_loss: 0.3967 - val_acc: 0.8256
Epoch 3/5
48878/48878 [==============================] - 5158s 106ms/step - loss: 0.3671 - acc: 0.8399 - val_loss: 0.3991 - val_acc: 0.8252
Epoch 4/5
48878/48878 [==============================] - 5047s 103ms/step - loss: 0.3585 - acc: 0.8460 - val_loss: 0.4013 - val_acc: 0.8253
Epoch 5/5
48878/48878 [==============================] - 5226s 107ms/step - loss: 0.3462 - acc: 0.8541 - val_loss: 0.4084 - val_acc: 0.8227


Through 5 epochs, the train set accuracy didn't get as high as the model without dropout, but validation accuracy didn't drop as low as the previous model. Even though the dropout added some generalization to the model, but the validation accuracy is still underperforming compared to logistic regression result.

There is another method I can try to prevent overfitting. By presenting the data in the same order for every epoch, there's a possibility that the model learns the parameters which also includes the noise of the training data, which eventually leads to overfitting. This can be improved by shuffling the order of the data we feed the model. Below I added shuffling to the batch generator function and tried with the same model structure and compared the result.

In [23]:
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0

In [24]:
%%time
model_s = Sequential()
model_s.add(Dense(64, activation='relu', input_dim=100000))
model_s.add(Dense(1, activation='sigmoid'))
model_s.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_s.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
48878/48878 [==============================] - 5394s 110ms/step - loss: 0.4081 - acc: 0.8143 - val_loss: 0.3935 - val_acc: 0.8256
Epoch 2/5
48878/48878 [==============================] - 5516s 113ms/step - loss: 0.3717 - acc: 0.8352 - val_loss: 0.3957 - val_acc: 0.8276
Epoch 3/5
48878/48878 [==============================] - 5291s 108ms/step - loss: 0.3611 - acc: 0.8418 - val_loss: 0.3978 - val_acc: 0.8258
Epoch 4/5
48878/48878 [==============================] - 5370s 110ms/step - loss: 0.3546 - acc: 0.8459 - val_loss: 0.4014 - val_acc: 0.8247
Epoch 5/5
48878/48878 [==============================] - 5242s 107ms/step - loss: 0.3509 - acc: 0.8480 - val_loss: 0.4038 - val_acc: 0.8261
CPU times: user 14h 14min 27s, sys: 3h 54min 36s, total: 18h 9min 4s
Wall time: 7h 26min 58s


The same model with non-shuffled training data had training accuracy of 87.36%, and validation accuracy of 79.78%. With shuffling, training accuracy decreased to 84.80% but the validation accuracy after 5 epochs has increased to 82.61%. It seems like the shuffling did improve the model's performance on the validation set. And another thing I noticed is that with or without shuffling also for both with or without dropout, validation accuracy tends to peak after 2 epochs, and gradually decrease afterwards.

Below I tried the same model with 20% dropout with shuffled data, this time only 2 epochs.

In [25]:
%%time
model_s_1 = Sequential()
model_s_1.add(Dense(64, activation='relu', input_dim=100000))
model_s_1.add(Dropout(0.2))
model_s_1.add(Dense(1, activation='sigmoid'))
model_s_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_s_1.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/2
48878/48878 [==============================] - 5131s 105ms/step - loss: 0.4102 - acc: 0.8139 - val_loss: 0.3963 - val_acc: 0.8237
Epoch 2/2
48878/48878 [==============================] - 5363s 110ms/step - loss: 0.3763 - acc: 0.8337 - val_loss: 0.3954 - val_acc: 0.8264
CPU times: user 5h 36min 14s, sys: 1h 33min 7s, total: 7h 9min 22s
Wall time: 2h 54min 55s


As same as the non-shuffled data, both the training accuracy and validation accuracy slightly dropped.

As I was going through the "deeplearning.ai" course by Andrew Ng, he states that the first thing he would try to improve a neural network model is tweaking the learning rate. I decided to follow his advice and try different learning rates with the model. Please note that except for the learning rate, the parameter for 'beta_1', 'beta_2', and 'epsilon' are set to the default values presented by the original paper "ADAM: A Method for Stochastic Optimization" by Kingma and Ba (2015). https://arxiv.org/pdf/1412.6980.pdf

In [36]:
%%time
import keras
custom_adam = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_2 = Sequential()
model_testing_2.add(Dense(64, activation='relu', input_dim=100000))
model_testing_2.add(Dense(1, activation='sigmoid'))
model_testing_2.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_2.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=2, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/2
48878/48878 [==============================] - 5278s 108ms/step - loss: 0.4107 - acc: 0.8133 - val_loss: 0.3953 - val_acc: 0.8250
Epoch 2/2
48878/48878 [==============================] - 5573s 114ms/step - loss: 0.3733 - acc: 0.8352 - val_loss: 0.3992 - val_acc: 0.8259
CPU times: user 5h 39min 43s, sys: 1h 35min 45s, total: 7h 15min 29s
Wall time: 3h 52s


In [37]:
%%time
custom_adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_3 = Sequential()
model_testing_3.add(Dense(64, activation='relu', input_dim=100000))
model_testing_3.add(Dense(1, activation='sigmoid'))
model_testing_3.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_3.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=2, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/2
48878/48878 [==============================] - 5189s 106ms/step - loss: 0.4126 - acc: 0.8125 - val_loss: 0.3966 - val_acc: 0.8238
Epoch 2/2
48878/48878 [==============================] - 5615s 115ms/step - loss: 0.3760 - acc: 0.8343 - val_loss: 0.3977 - val_acc: 0.8261
CPU times: user 5h 37min 39s, sys: 1h 34min 58s, total: 7h 12min 37s
Wall time: 3h 4s


In [38]:
%%time
custom_adam = keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_4 = Sequential()
model_testing_4.add(Dense(64, activation='relu', input_dim=100000))
model_testing_4.add(Dense(1, activation='sigmoid'))
model_testing_4.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_4.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=2, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/2
48878/48878 [==============================] - 5434s 111ms/step - loss: 0.4739 - acc: 0.7801 - val_loss: 0.4761 - val_acc: 0.7568
Epoch 2/2
48878/48878 [==============================] - 5338s 109ms/step - loss: 0.4844 - acc: 0.7748 - val_loss: 0.5953 - val_acc: 0.7294
CPU times: user 5h 33min 31s, sys: 1h 35min 4s, total: 7h 8min 35s
Wall time: 2h 59min 33s


In [40]:
%%time
custom_adam = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_5 = Sequential()
model_testing_5.add(Dense(64, activation='relu', input_dim=100000))
model_testing_5.add(Dense(1, activation='sigmoid'))
model_testing_5.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_5.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
48878/48878 [==============================] - 5215s 107ms/step - loss: 0.4081 - acc: 0.8147 - val_loss: 0.3942 - val_acc: 0.8234
Epoch 2/5
48878/48878 [==============================] - 5097s 104ms/step - loss: 0.3713 - acc: 0.8352 - val_loss: 0.3945 - val_acc: 0.8261
Epoch 3/5
48878/48878 [==============================] - 5093s 104ms/step - loss: 0.3614 - acc: 0.8413 - val_loss: 0.3983 - val_acc: 0.8252
Epoch 4/5
48878/48878 [==============================] - 5102s 104ms/step - loss: 0.3555 - acc: 0.8449 - val_loss: 0.4003 - val_acc: 0.8262
Epoch 5/5
48878/48878 [==============================] - 5176s 106ms/step - loss: 0.3515 - acc: 0.8473 - val_loss: 0.4022 - val_acc: 0.8258
CPU times: user 13h 53min 56s, sys: 3h 49min 39s, total: 17h 43min 36s
Wall time: 7h 8min 10s


Having tried four different learning rates (0.0005, 0.005, 0.01, 0.1), none of them outperformed the default learning rate of 0.001.

Maybe I can try to increase the number of hidden nodes, and see how it affects the performance. Below model has 128 nodes in the hidden layer.

In [27]:
%%time
model_s_2 = Sequential()
model_s_2.add(Dense(128, activation='relu', input_dim=100000))
model_s_2.add(Dense(1, activation='sigmoid'))
model_s_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_s_2.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=2, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/2
48878/48878 [==============================] - 9391s 192ms/step - loss: 0.4083 - acc: 0.8141 - val_loss: 0.3959 - val_acc: 0.8228
Epoch 2/2
48878/48878 [==============================] - 9173s 188ms/step - loss: 0.3717 - acc: 0.8354 - val_loss: 0.3942 - val_acc: 0.8284
CPU times: user 12h 23min 45s, sys: 2h 56min 41s, total: 15h 20min 27s
Wall time: 5h 9min 30s


With 128 hidden nodes, validation accuracy got close to the performance of logistic regression. I could experiment further with increasing the number of hidden layers, but for the above 2 epochs to run, it took 5 hours. Considering that logistic regression took less than a minute to fit, even if the neural network can be improved further, this doesn't look like an efficient way.

Below is a table with all the results I got from trying different models above. Please note that I have compared performance at 2 epochs since some of the models only ran for 2 epochs.

| model | learning rate | input layer (nodes) | data shuffling | hidden layer (nodes) | dropout | output layer (nodes) | training accuracy | validation accuracy |
|-----|------|---|-----|----|----|----|----|
| ANN_1 | 0.001 | 1 (100,000)  | X | 1 (64) relu  |  X  | 1 (1) sigmoid   | 83.52% | 82.54% |
| ANN_2 | 0.001 | 1 (100,000)  | X | 1 (64) relu  |  0.2  | 1 (1) sigmoid   | 83.35% | 82.56% |
| ANN_3 | 0.001 | 1 (100,000)  | O | 1 (64) relu  |  X  | 1 (1) sigmoid   | 83.52% | 82.76% |
| ANN_4 | 0.001 | 1 (100,000)  | O | 1 (64) relu  |  0.2  | 1 (1) sigmoid   | 83.37% | 82.64%  |
| ANN_5 | 0.0005 | 1 (100,000)  | O | 1 (64) relu  |  X  | 1 (1) sigmoid   | 83.52% | 82.61%  |
| ANN_6 | 0.005 | 1 (100,000)  | O | 1 (64) relu  |  X  | 1 (1) sigmoid   | 83.52% | 82.59%  |
| ANN_7 | 0.01 | 1 (100,000)  | O | 1 (64) relu  |  X  | 1 (1) sigmoid   | 83.43% | 82.61%  |
| ANN_8 | 0.1 | 1 (100,000)  | O | 1 (64) relu  |  X  | 1 (1) sigmoid   | 77.48% | 72.94%  |
| ANN_9 | 0.001 | 1 (100,000)  | O | 1 (128) relu  |  X  | 1 (1) sigmoid   | 83.54% | 82.84%  |

Except for ANN_8 (with a learning rate of 0.1), the model performance only varies in the decimal place, and the best model is ANN_9 (with one hidden layer of 128 nodes) at 82.84% validation accuracy.

As a result, in this particular case, neural network models failed to outperform logistic regression. This might be due to the high dimensionality and sparse characteristics of the textual data. I have also found a research paper, which compared model performance with high dimension data. According to "An Empirical Evaluation of Supervised Learning in High Dimensions" by Caruana et al.(2008), logistic regression showed as good performance as neural networks, in some cases outperforms neural networks. http://icml2008.cs.helsinki.fi/papers/632.pdf

Through all the trials above I learned some valuable lessons. Implementing and tuning neural networks is a highly iterative process and includes many trials and errors. Even though the neural network is a more complex version of logistic regression, it doesn't always outperform logistic regression, and sometimes with high dimension sparse data, logistic regression can deliver good performance with much less computation time than neural network.

In the next post, I will implement a neural network with Doc2Vec vectors I got from the previous post. Hopefully, with dense vectors such as Doc2Vec, a neural network might show some boost. Fingers crossed.